# **Camada Silver (Cleaned / Refined Layer)**

Dados já limpos, validados e com estruturação melhorada.

Normalmente inclui tratamento de erros, remoção de duplicatas e padronização.

Serve para análises intermediárias e alimentação de modelos.

In [0]:
%sql
SELECT current_catalog();

In [0]:
spark.sql("USE CATALOG pipe1")

# Tratamento de dados da LCB

In [0]:
from pyspark.sql.functions import col, date_format, round

# Ler tabela Bronze
df_bronze = spark.table("pipe1.bronze.brz_despesas_lcb_contemporaneo")

# 1. Remover a coluna _line
df = df_bronze.drop("_line")

# 3. Transformar _fivetran_synced para yyyy-MM-dd e timestamp (mantendo a coluna original, opcional)
df = df.withColumn("_fivetran_synced", date_format(col("_fivetran_synced"), "yyyy-MM-dd'T'HH:mm:ss"))

# 3. Converter 'periodo' para string ano-mês
df = df.withColumn("periodo", date_format(col("periodo"), "yyyy-MM"))

# 4. Arredondar kwh_dia para 2 casas decimais
df = df.withColumn("kwh_dia", round(col("kwh_dia"), 2))

# 5. Arredondar total_agua para 2 casas decimais
df = df.withColumn("total_agua", round(col("total_agua"), 2))

# 6. Substituir todos os valores nulos por 0
df = df.fillna(0)

# 7. Salvar na camada Silver
df.write.format("delta").mode("overwrite").saveAsTable("pipe1.silver.slv_despesas_lcb_contemporaneo")






In [0]:
# dropar a tabela
#spark.sql("DROP TABLE IF EXISTS pipe1.silver.despesas_lcb_contemporaneo")



In [0]:
df_silver = spark.table("pipe1.silver.slv_despesas_lcb_contemporaneo")
display(df_silver)

# Tratamento de dados da Timbauvas

In [0]:
%sql
select * from workspace.google_drive.dados_despesas_home_despesa_timbauvas

In [0]:
from pyspark.sql.functions import col, to_date, to_timestamp, date_format, round, coalesce, lit,  date_format
from pyspark.sql.types import DoubleType

# 1. Carregar dados da camada Bronze
df = spark.table("pipe1.bronze.brz_despesas_timbauvas")

# 2. Remover a coluna _line
df = df.drop("_line")

# 3. Transformar _fivetran_synced para yyyy-MM-dd e timestamp (mantendo a coluna original, opcional)
df = df.withColumn("_fivetran_synced", date_format(col("_fivetran_synced"), "yyyy-MM-dd'T'HH:mm:ss"))

# 4. Alterar a coluna 'periodo' para ano-mês (string)
df = df.withColumn("periodo", date_format(to_timestamp(col("periodo")), "yyyy-MM"))

# 5. Arredondar kwh_dia para 2 casas decimais
df = df.withColumn("kwh_dia", round(col("kwh_dia"), 2))

# 6. Converter despesa_de_luz para número e 2 casas decimais
df = df.withColumn("despesa_de_luz", round(col("despesa_de_luz").cast(DoubleType()), 2))

# 7. Substituir todos os valores nulos por 0
df = df.fillna(0)

# 8. Arredondar consumo_kg para 2 casas decimais
df = df.withColumn("consumo_kg", round(col("consumo_kg"), 2))

# 9. Arredondar total_agua para 2 casas decimais
df = df.withColumn("total_agua", round(col("total_agua"), 2))

# 10. Gravar na camada Silver
df.write.format("delta").mode("overwrite").saveAsTable("pipe1.silver.slv_despesas_timbauvas")


In [0]:
df_silver = spark.table("pipe1.silver.slv_despesas_timbauvas")
display(df_silver)

In [0]:
# dropar a tabela
# spark.sql("DROP TABLE IF EXISTS pipe1.silver.slv_despesas_timbauvas")